Detección de caras con webcam

In [2]:
import cv2
import time
import FaceNormalizationUtils as faceutils

In [3]:
from imutils import face_utils
import imutils
import numpy as np
import dlib

In [4]:
def FaceDetectionDNN(img):
    conf_threshold = 0.7
    faceProto = "opencv_face_detector.pbtxt"
    faceModel = "opencv_face_detector_uint8.pb"
    faceNet = cv2.dnn.readNet(faceModel, faceProto)
    # Detecta objetos
    frameHeight = img.shape[0]
    frameWidth = img.shape[1]
    blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [104, 117, 123], True, False)

    faceNet.setInput(blob)
    detections = faceNet.forward()
    faces = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            faces.append([x1, y1, x2 - x1, y2 - y1])
    return faces

In [5]:
def getLargest(objects):
    if len(objects) < 1:
        return -1
    elif len(objects) == 1:
        return 0
    else:
        areas = [w * h for x, y, w, h in objects]
    return np.argmax(areas)

In [6]:
def GetFacialLandmarks(img,roi,model=0):
    p = "shape_predictor_5_face_landmarks.dat"
    predictor = dlib.shape_predictor(p)
    p = "shape_predictor_68_face_landmarks.dat"
    predictor68 = dlib.shape_predictor(p)
    if model == 0:
        points = predictor(img, roi)
    else:
        points = predictor68(img, roi)

    if points is not None:
            # ibujamos recuadro mayor
        shape = face_utils.shape_to_np(points)
            # for (x, y) in shape:
            #     cv2.circle(imagenRGB, (x, y), 2, (255, 255, 255), -1)

        left_eye_x = int(points.part(3).x - abs(points.part(3).x - points.part(2).x) / 2.)
        left_eye_y = int(points.part(3).y - abs(points.part(3).y - points.part(2).y) / 2.)
        right_eye_x = int(points.part(1).x + abs(points.part(1).x - points.part(0).x) / 2.)
        right_eye_y = int(points.part(1).y - abs(points.part(1).y - points.part(0).y) / 2.)

        nose_x = int(points.part(4).x)
        nose_y = int(points.part(4).y)

        if right_eye_x - left_eye_x == 0:
            return None
        m1 = (right_eye_y - left_eye_y) / (right_eye_x - left_eye_x)
        if m1 != 0:
            m2 = -1 / m1
            b2 = nose_y - m2 * nose_x
            x_c = ((nose_y + 1.5 * abs(points.part(3).x - points.part(2).x) / 2.) - b2) / m2
            y_c = m2 * x_c + b2

            b3 = y_c - m1 * x_c
            left_mouth_x = ((y_c + m1 * 2 * abs(points.part(3).x - points.part(2).x) / 2.) - b3) / m1
            left_mouth_y = m1 * left_mouth_x + b3
            right_mouth_x = ((y_c - m1 * 2 * abs(points.part(3).x - points.part(2).x) / 2.) - b3) / m1
            right_mouth_y = m1 * right_mouth_x + b3
        else:
            left_mouth_x = points.part(4).x + 1.5 * abs(points.part(3).x - points.part(2).x) / 2.
            left_mouth_y = nose_y + abs(points.part(3).x - points.part(2).x) / 2.
            right_mouth_x = (points.part(4).x) - 1.5 * abs(points.part(3).x - points.part(2).x) / 2.
            right_mouth_y = nose_y + abs(points.part(3).x - points.part(2).x) / 2.

        points = [left_eye_x, right_eye_x, nose_x, left_mouth_x, right_mouth_x, left_eye_y, right_eye_y, nose_y,
                      left_mouth_y, right_mouth_y]


        return [points], shape
    else:
        return None

In [7]:
def DetectLargestFaceEyesDNN(img,eyesdet):
    # Face detection DNN
    faces = FaceDetectionDNN(img)

    # Makes us of the largest face
    iface = getLargest(faces)
    if iface >= 0:
        (x, y, w, h) = faces[iface]
        # Composes rectangle
        face = dlib.rectangle(left=x, top=y, right=x + w, bottom=y + h)

        if eyesdet == 'DLIB5':
            values = GetFacialLandmarks(img, face)

            if values is not None:
                points, shape = values

                # right eye n the image positions 0 ans 1 (left 2 and 3)
                re = np.mean(shape[0:2], 0)
                le = np.mean(shape[2:4], 0)

                return [x, y, w, h], [le[0], le[1], re[0], re[1]], shape

            else:
                return [-1, -1, -1, -1], [], []

        elif eyesdet == 'DLIB68':
            values = GetFacialLandmarks(img, face, 1)

            if values is not None:
                points, shape = values

                le = np.mean(shape[36:42], 0)
                re = np.mean(shape[42:48], 0)

                return [x, y, w, h], [le[0], le[1], re[0], re[1]], shape
            else:
                return [-1, -1, -1, -1], [], []

        else:
            return [-1, -1, -1, -1], [], []

    else:
        return [-1, -1, -1, -1], [], []

In [8]:
normalizatorHS = faceutils.Normalization()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX
image=cv2.imread('mask.png', cv2.IMREAD_UNCHANGED)
# Webcam connection
cap = cv2.VideoCapture(0)

#Set camera resolution
cap.set(3,400);
cap.set(4,400);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = DetectLargestFaceEyesDNN(frame, 'DLIB5')
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face

        #Si existe cara
        if x > -1:
            #Ajustar tamaño máscara
            resized_image = imutils.resize(image, width=w, height=h)
            row_image = resized_image.shape[0]

            #Fragmeto de imagen donde se va ha aplicar el filtro
            n_frame = frame[y:y+row_image,x:x+w]

            #Creación de las máscaras
            mask = resized_image[:, :, 3] 
            mask_inv = cv2.bitwise_not(mask)
            
            #Fragmeto de imagen con filtro a color y con el fondo negro
            bg_black = cv2.bitwise_and(resized_image, resized_image, mask=mask)
            bg_black = bg_black[:, :, 0:3]
            
            #Fragmeto de imagen con filtro en negro y fondo de color
            bg_frame = cv2.bitwise_and(n_frame, n_frame, mask=mask_inv)

            #Add de fondo negro y filtro negro para obtener imagen final
            result = cv2.add(bg_black, bg_frame)

            #Añadimos el resultado al frame
            frame[y:y+row_image,x:x+w] = result

    # Show resulting image
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()